# CRC Fehlererkennung

Zuerst erstellen wir eine Klasse zur Simulation eines BSC.

In [1]:
from math import ceil


class BitArray:
    def __init__(self, bits=None, length=None, file=None):
        if bits is None:
            self.array = 0
            self.length = 0
            if file is not None:
                self.write(file)
        elif type(bits) in {chr, str}:
            self.array = int(bits, 2)
            self.length = len(bits)
            if file is not None:
                self.write(file)
        elif type(bits) == int:
            self.array = bits
            self.length = length if length else bits.bit_length()
            if file is not None:
                self.write(file)
        elif type(bits) == BitArray:
            self.array = bits.array
            self.length = bits.length
            if file is not None:
                self.write(file)
        elif type(bits) == bytes:
            temp = BitArray(int.from_bytes(bits, 'big'))
            cutoff = int(temp[:8])
            self.array = int(temp[8 + cutoff:])
            self.length = len(temp) - (8 + cutoff)
            del temp
            if file is not None:
                self.write(file)
        elif bits is None and file is not None:
            try:
                self.__init__(self.read(file))
            except:
                raise Exception(f'File {file} is invalid!')
        else:
            raise Exception(f"""
Bits must be of type str or int.
{type(bits)} is something completly different.
""")
    
    def __call__(self, step=1):
        for i in range(0, len(self), step):
            if i+step <= len(self):
                yield self[i:i+step]
    
    def __iter__(self):
        for i in range(len(self)):
            yield self[i]
    
    def __getitem__(self, idx):
        if type(idx) == slice:
            start = (idx.start if idx.start is not None else 0) % len(self)
            stop = (idx.stop if idx.stop is not None else len(self)) % (len(self) + 1)
            return BitArray((self.array >> len(self) - stop) & ((1 << stop - start) - 1),
                            length=stop-start)
        else:
            return BitArray((self.array & 1 << (len(self) - idx - 1)) >> (len(self) - idx - 1) , length=1)
    
    def __setitem__(self, idx, value):
        if type(value) != BitArray:
            raise Exception(f"""
Value must be of type BitArray.
{type(value)} is something completly different.
""")
        cutoff = min(len(self), idx + len(value))
        self.array = (self[:idx]|value[:cutoff]|self[cutoff:]).array
    
    def __or__(self, other):
        return BitArray(self.array << len(other)|other.array, length=len(self) + len(other))
    
    def __xor__(self, other):
        if isinstance(other, BitArray):
            return BitArray(self.array ^ other.array, length=max(len(self), len(other)))
        else:
            return BitArray(self.array ^ other, length=max(len(self), other.bit_length()))
    
    def __repr__(self):
        return f'BitArray[{str(self)}]'
    
    def __str__(self):
        if self.array is not None:
            return bin(self.array).replace('0b', '').zfill(len(self))[:len(self)]
        else:
            return ''
    
    def bytes(self):
        return (BitArray(-len(self) % 8, length=8)|self).array.to_bytes(ceil(len(self)/8) + 1, 'big')
    
    def __len__(self):
        return self.length
    
    def __int__(self):
        return self.array
    
    def __bool__(self):
        return bool(self.array)
    
    def __eq__(self, other):
        return self.array == other.array and len(self) == len(other)
    
    def __lshift__(self, idx):
        return BitArray(self.array << idx, length=self.length + idx)
    
    def __rshift__(self, idx):
        return BitArray(self.array >> idx, length=max(self.length - idx, 0))

In [2]:
from random import uniform


class BSC:
    def __init__(self, p):
        self.p = p
    
    def error(self, n):
        return BitArray(''.join('0' if uniform(0, 1) > self.p else '1' for _ in range(n)))
    
    def __call__(self, m):
        return self.error(len(m)) ^ m
    

channel = BSC(0.2)
X = BitArray('101010101010101010101010')
Y = channel(X)

print(f"""
X:
{X}
Y:
{Y}
ERROR:
{X^Y}
""")


X:
101010101010101010101010
Y:
101010010010101010100011
ERROR:
000000111000000000001001



### Was geschiet bei der Übertragung der Morsekodierung aus PK1

Da die Blöcke für jeden Buchstaben in meiner Lösung für das PK1 unterschiedliche Längen haben und diese sequenziell verarbeitet werden müssen, um zu bestimmen wo der letzte Block endet und wo der nächte beginnt, wird die Dekodierung nicht mehr funktionieren, sobald ein Bitfehler im Präfix (welcher die Blocklänge beinhaltet) auftaucht. Ein Bitfehler im Contentpart des Blocks dahingegen würde nur zu einer falschen dekodierung des Blocks führen, die restlichen Blöcke aber nicht beeinflussen. Sobald ein präfix korumpiert ist, wird der Aufteilung der Blöcke asynchron und das Dekodierungsprogramm wird foraussichtlich nicht terminieren. Der häufigste Fehler wird wohl der sein, dass auf die Bedeutung des Präfixes `111` zugefriffen soll, welcher für den Code jedoch nicht definiert ist. Daher stammt der fast immer auftretende `KeyError: 7`

In [3]:
class Encoder:
    def __init__(self):
        self.morse = {
            'A': '._', 'B': '_...', 
            'C': '_._.', 'D': '_..', 
            'E': '.', 'F': '.._.', 
            'G': '__.', 'H': '....', 
            'I': '..', 'J': '.___', 
            'K': '_._', 'L': '._..', 
            'M': '__', 'N': '_.', 
            'O': '___', 'P': '.__.', 
            'Q': '__._', 'R': '._.', 
            'S': '...', 'T': '_', 
            'U': '.._', 'V': '..._', 
            'W': '.__', 'X': '_.._', 
            'Y': '_.__', 'Z': '__..', 
            '0': '_____', '1': '.____', 
            '2': '..___', '3': '...__', 
            '4': '...._', '5': '.....', 
            '6': '_....', '7': '__...', 
            '8': '___..', '9': '____.', 
            ' ': ':', '.': '*',
        }
        self.morse_inv = {v: k for k, v in self.morse.items()}
    
    def encode(self, x, status=False):
        if set(x) - {'.', '_', ':', '*'}:
            x = self.encode_morse(x)
            if status: print(x)
        x = self.encode_binary(x)
        return x

    def decode(self, x, status=False):
        if isinstance(x, BitArray):
            x = self.decode_binary(x)
            if status: print(x)
        x = self.decode_morse(x)
        return x
    
    def encode_morse(self, text):
        return ' '.join(self.morse[char] for char in text.upper() if char in self.morse)

    def decode_morse(self, text):
        return ''.join(self.morse_inv[code] for code in text.split(' ') if code in self.morse_inv)


class EncoderThree(Encoder):
    def __init__(self):
        super().__init__()
        self.prefix = {
            (True, 1, None): 0,
            (True, 2, None): 1,
            (True, 3, None): 2,
            (True, 4, None): 3,
            (True, 5, None): 4,
            (False, 1, ':'): 5,
            (False, 1, '*'): 6,
        }
        self.prefix_inv = {v: k for k, v in self.prefix.items()}

    def encode_binary(self, codes):
        stream = BitArray()
        for x in codes.split(' '):
            stream = stream|BitArray(self.prefix[
                not bool(set(x)-{'_', '.'}),
                len(x),
                x if (punctuation := x in {'*', ':'}) else None,
            ], length=3)
            if not punctuation:
                stream = stream|self.phi(x)
        return stream

    def decode_binary(self, stream):
        chars, i = [], 0
        while i < len(stream):
            letter, char_length, punctuation = self.prefix_inv[int(stream[i:i+3])]
            if letter:
                chars.append(''.join(self.phi_inv(stream[i+3:i+3+char_length])))
                i += 3 + char_length 
            else:
                chars.append(punctuation)
                i += 2 + char_length
        return ' '.join(chars)
    
    def phi(self, text):
        b = BitArray()
        for char in text:
            if char == '.':
                b = b|BitArray('0')
            elif char == '_':
                b = b|BitArray('1')
        return b
    
    def phi_inv(self, stream):
        return ''.join(['.', '_'][int(bit)] for bit in stream())

In [4]:
bricks = "Basically I belive in peace and smashing two bricks together."

three = EncoderThree()
stream = three.encode(bricks, True)

corrupted_stream = channel(stream)

text = three.decode(corrupted_stream)
print(text)

_... ._ ... .. _._. ._ ._.. ._.. _.__ : .. : _... . ._.. .. ..._ . : .. _. : .__. . ._ _._. . : ._ _. _.. : ... __ ._ ... .... .. _. __. : _ .__ ___ : _... ._. .. _._. _._ ... : _ ___ __. . _ .... . ._. *


KeyError: 7

# Galois Felder
Das Galois Feld ist eine Klasse von Körpern, welches definiert wird als $GF_{p^n}[x]$. Dabei ist $p \in \mathbb{P}$ (also prim) und $n \in \mathbb{N}$. Alle Elemende aus dem Körper werden dargestellt, als ein Polynom der Variable $x$. $n$ gibt dann den maximalen Grad aller Polynome an, und $p$ das Modul der Koeffizienten. Ein beispielhaftes Element aus $GF_{3^5}[x]$ wäre $2x^5 + 0x^4 + 1x^3 + 1x^2 + 0x^1 + 2x^0$.

### Addition
Die Additionsverknüpfung zweier Elemente ist die Addition der Polynome, wobei jeder Koeffizient im Modul $p$ verrechnet wird. 
$$a, b \in GF_{p^n}: a + b = \sum_{i=0}^n (a_i + b_i\, \text{mod} \, p )  x^i $$

### Multiplikation
Die Multiplikation wird definiert als Polynommultiplikation mit anschließender Reduktion $a \cdot b \,\text{mod} \, g$ wobei $g$ das Generatorpolynom des Galoisfeldes ist. Für jedes $GF$ gibt es mindestens ein Generatorpolynom, welches die Bedingungen für den Körper erfüllt. (Assiziativität von Multiplikation und Addition, Inverses Element für Addition und Multiplikation, Distributivität etc...)


# CRC
Die Fehlererkennung in CRC ist definiert im $GF_{2^n}[x]$. Dabei wird ein Element des Körpers als Polynom dargestellt. Für eine Bitfolge $c$ der länge $N$ ist damit die korresponierende Darstellung in unserem Galoisfeld: $$c(x) = \sum_{i=0}^n c_ix^i \in GF_{2^n}[x]$$

### Implementierung
Da unser $p=2$ ist, können wir die Addition einfach als XOR verkünpfung implementieren. Die Multiplikation wird als Addition (also XOR) von Verschiebungen um die koefizienten des zweiten Elementes implementiert. Die Reduktion durch $g$ wird dann wie im Skript umgesetzt.

In [5]:
class CRC:
    def __init__(self, g, k):
        self.g = g
        self.k = k
    
    def add(self, a, b):
        return a ^ b
    
    def mul(self, a, b):
        result = BitArray()
        for i, v in enumerate(a):
            i = len(a) - i - 1
            result = result ^ (int(v) * int(b) << i)
        return result
    
    def mod(self, a):
        result = BitArray()
        rest = BitArray(a)
        for i, v in enumerate(rest[:-len(self.g)]):
            result <<= 1
            result = result ^ v
            rest = rest ^ (int(rest[i]) * int(self.g) << (len(a) - len(self.g) - i))
        rest.length = self.g.length - 1
        return rest

    def p(self, s):
        return self.mod(s<<(len(self.g)-1))
    
    def c(self, s):
        return self.add(self.p(s), s<<(len(self.g)-1))
    
    def secure(self, stream):
        output = BitArray()
        for s in stream(self.k):
            output = output|self.c(s)
        return output
    
    def check(self, stream):
        return all(not self.mod(c) for c in stream(self.k + len(self.g) - 1)) 

CRC_8 = BitArray('111010101') # Bluetooth


gf = CRC(CRC_8, 24)


c = gf.secure(stream)
corrupted = channel(c)
print(f"""
Original:         {stream}

CRC_Code:         {c}

Corrupted:        {corrupted}

Error:            {c^corrupted}

check(c):         {gf.check(c)}
check(corrputed): {gf.check(corrupted)} 
""")


Original:         01110000010101000000100011101000101011010001101000111011101001001010111000000001101000010001100010000101001000011010101101100000001010111010000010100101001100101001010100000011100101010000011000000100001100101101010001010011010111101011100001001000100011101001010101000010100010101110101100000000101100000000010010110

CRC_Code:         01110000010101000000100001000101111010001010110100011010000111010011101110100100101011100100110000000001101000010001100011101101100001010010000110101011010011100110000000101011101000001110110110100101001100101001010110001111000000111001010100000110111011010000010000110010110101001011111001010011010111101011100001110000010010001000111010010101110111010100001010001010111010110000011100000000101100000000010001010000

Corrupted:        0110000100011111000110000100011111111000101011011000000100011101001100111011010010001110001011000000001110100000001010001100110110100101011100010010100111001100111010010001111100101010111011010000011100110010

In [6]:
CRC_TEST = BitArray('1011')


Data = BitArray('10111')
Data_2 = BitArray('11001')
Data_3 = BitArray('10101011010010001')

CRC_P = BitArray('100101')

gf = CRC(CRC_P, len(Data_3))
print(gf.secure(Data_3))

1010101101001000111110
